In [ ]:
import $ivy.`org.apache.spark::spark-sql:2.4.0` // Or use any other 2.x version here
import $ivy.`com.markblokpoel::lanag-core:0.3.5`
import $ivy.`com.lihaoyi::scalatags:0.6.7`
interp.load.cp(new java.net.URL("https://github.com/markblokpoel/lanag-ambiguityhelps/blob/jupyternotebooks/binaries/lanag-ambiguityhelps-0.1.jar?raw=true"))

In [ ]:
import org.apache.spark.sql._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import com.markblokpoel.lanag.ambiguityhelps.experiments.uniform.UniformExperiment
import scalatags.Text.all._
import java.time.Instant
import java.io.File

In [ ]:
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

val data = UniformExperiment.run(
    spark,
    vocabularySize = 8,
    contextSize = 4,
    changeResolution = 0.2,
    sampleSize = 10,
    interactionLength = 30,
    beta = Double.PositiveInfinity,
    randomSeed = 9064    
)

val flatData = UniformExperiment.flattenData(spark, data)
val outputFolder = "output/rsa_uniform-"+Instant.now.toEpochMilli
flatData.write.mode(SaveMode.Overwrite).option("header", value = true).csv(outputFolder)

In [ ]:
Html(
    p(h1("Your files are ready to download:"), br(),
        (for(file <- new File(outputFolder).list.filter(f => f.endsWith(".csv"))) yield Seq(
            a(href:=s"$outputFolder/$file", `type`:="text/csv", attr("download"):=s"$outputFolder/$file", s"$outputFolder/$file"),br()
        ))).render
)